In [4]:
%%capture 
%run ./minimal_blockchain.ipynb

## SPHINCS+

For SPHINCS+, there exists a Python package called pyspx.

Documentation is available here:
- https://github.com/sphincs/pyspx
- https://pypi.org/project/PySPX/

pyspx provides capabilites for generating public-private key pairs, signing and verifying messages.

Example usage: https://github.com/nakov/Practical-Cryptography-for-Developers-Book/blob/3cbe60554b9a1fce016a41fce708525adcd0323f/quantum-safe-cryptography/quantum-safe-signatures-example.md


SPHINCS+ uses tweakable hash functions. The official document (https://sphincs.org/data/sphincs+-specification.pdf) defines three different signature schemes which are obtained by instantiating the cryptographic function families of SPHINCS+ with SHA-256, SHAKE256, and Haraka.

The following implementation uses SHAKE256, as there is existing support in pyspx.

In [5]:
pip install pyspx


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.1_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pyspx.shake_128f as sphincs  # Assuming SPHINCS+ shake256 with 128-bit security

class SphincsProvider(CryptoProvider):
    """
    SphincsProvider implements the SPHINCS+ cryptographic operations for key generation,
    signing, signature verification, and hashing using the post-quantum secure SPHINCS+ method.
    """

    def __repr__(self) -> str:
        return "SphincsProvider"

    def generate_key_pair(self) -> Tuple[bytes, bytes]:
        """
        Generate a SPHINCS+ key pair.

        Returns:
            Tuple[bytes, bytes]: A tuple containing the private and public keys as bytes.
        """
        seed = os.urandom(sphincs.crypto_sign_SEEDBYTES)
        public_key, private_key = sphincs.generate_keypair(seed)
        return private_key, public_key

    def sign(self, private_key: bytes, message: str) -> bytes:
        """
        Sign a message using the provided SPHINCS+ private key.

        Args:
            private_key (bytes): The private key used to sign the message.
            message (str): The message to be signed.

        Returns:
            bytes: The generated signature as bytes.
        """
        message_bytes = message.encode('utf-8')
        signature = sphincs.sign(message_bytes, private_key)
        return signature

    def verify(self, public_key: bytes, message: str, signature: bytes) -> bool:
        """
        Verify a signature for a given message and SPHINCS+ public key.

        Args:
            public_key (bytes): The public key used to verify the signature.
            message (str): The message whose signature is to be verified.
            signature (bytes): The signature to verify.

        Returns:
            bool: True if the signature is valid; False otherwise.
        """
        message_bytes = message.encode('utf-8')
        try:
            return sphincs.verify(signature, message_bytes, public_key)
        except Exception:
            return False

    def hash(self, data: str) -> str:
        """
        Compute a cryptographic hash of the provided data using SHAKE-256.

        Args:
            data (str): The data to be hashed.

        Returns:
            str: The resulting hash as a hexadecimal string.
        """
        hasher = hashlib.shake_256()
        hasher.update(data.encode('utf-8'))
        return hasher.hexdigest(64)  # 64 hex digits for 256-bit output


In [7]:
crypto_provider = SphincsProvider()
private_key1, public_key1 = crypto_provider.generate_key_pair()
address1 = base64.b64encode(public_key1).decode("utf-8")
private_key2, public_key2 = crypto_provider.generate_key_pair()
address2 = base64.b64encode(public_key2).decode("utf-8")

transaction1 = Transaction(address1, address2, 30, crypto_provider)
transaction1.sign_transaction(private_key1)

transaction2 = Transaction(address2, address1, 10, crypto_provider)
transaction2.sign_transaction(private_key2)

transaction3 = Transaction(address2, address1, 15, crypto_provider)
transaction3.sign_transaction(private_key2)

blockchain = Blockchain(2, 3, crypto_provider)
blockchain.add_transaction(transaction1)
blockchain.add_transaction(transaction2)
blockchain.add_transaction(transaction3)
blockchain.mine_pending_transactions()

print(blockchain)

print(f"Balance of address1({address1}): {blockchain.get_balance(address1)}")
print(f"Balance of address2({address2}): {blockchain.get_balance(address2)}")

blockchain.mine_pending_transactions()

print(f"Balance of address1({address1}): {blockchain.get_balance(address1)}")
print(f"Balance of address2({address2}): {blockchain.get_balance(address2)}")

print(f"Blockchain validity: {blockchain.is_valid()}")
blockchain.chain[1].transactions[1].amount = 20
print(f"Blockchain validity after modification: {blockchain.is_valid()}")


private_key1_size = sys.getsizeof(private_key1)
public_key1_size = sys.getsizeof(public_key1)
print(f"Private Key 1 Size: {private_key1_size} bytes")
print(f"Public Key 1 Size: {public_key1_size} bytes")

transaction1_signature_size = sys.getsizeof(transaction1.signature)
print(f"Transaction 1 Signature Size: {transaction1_signature_size} bytes")

blockchain_data = json.dumps([block.__dict__ for block in blockchain.chain], default=str).encode()
blockchain_size = sys.getsizeof(blockchain_data)
print(f"Blockchain Storage Size: {blockchain_size / 1024:.2f} KB")

Blockchain with Blocksize 2 and Difficulty 3

Block #0
Previous Hash: 0
Transactions:
Timestamp: 1732807165.375974
Nonce: 0
Hash: 6bce8535252ce949827ad140d42dabcf64cee6a8e6255c8aaa1dcba804708045b21fac4e440213b2db2e1214cc953faa7a726c8706d442c6f56f3a021ca9b7c6

Block #1
Previous Hash: 6bce8535252ce949827ad140d42dabcf64cee6a8e6255c8aaa1dcba804708045b21fac4e440213b2db2e1214cc953faa7a726c8706d442c6f56f3a021ca9b7c6
Transactions:
  | Sender:     5n/QNjH0rXINCd4/Rv2kpvTvbbC/73+ef/bqflI2L2Y=
  | Recipient:  mPnXnYky5QJ8eBkdhjbDVocqPRXgPlAod/z4F/7o34I=
  | Amount:     30
  | Signature:  ...
  | 
  | Sender:     mPnXnYky5QJ8eBkdhjbDVocqPRXgPlAod/z4F/7o34I=
  | Recipient:  5n/QNjH0rXINCd4/Rv2kpvTvbbC/73+ef/bqflI2L2Y=
  | Amount:     10
  | Signature:  ...
Timestamp: 1732807165.37605
Nonce: 3822
Hash: 00030067233a865b7525acec3adfe9640f05aaabafcd3a776526beb7d988fee935c66d64f5ef54d79f57a77c31a7e4a34dd9cc4800ce6af41bfee7233e45132f

Pending Transactions:
  | Sender:     mPnXnYky5QJ8eBkdhjbDVocqPRXgPlAo